In [283]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
import datetime
warnings.filterwarnings('ignore')

In [284]:
data=yf.download('TSLA',start='2023-01-01',end='2024-01-01')
data.to_csv('_data1_.csv')
data=pd.read_csv('_data1_.csv')

[*********************100%%**********************]  1 of 1 completed


In [285]:
data['Heikin Close']=(data['Open']+data['Close']+data['Low'] + data['High'])/4
data['Heikin Open']=(data['Open'].shift(1)+data['Close'].shift(1))/2
data['H-High']=data[['High','Heikin Open','Heikin Close']].max(axis=1)
data['L-Low']=data[['Low','Heikin Open','Heikin Close']].min(axis=1)

In [286]:
def MACD(data):
    data['MACD']=data['Heikin Close'].ewm(span=12,adjust=False).mean()-data['Heikin Close'].ewm(span=26,adjust=False).mean()
    data['Signal']=data['MACD'].ewm(span=9,adjust=False).mean()
    return data

In [287]:
data=MACD(data)

In [288]:
S1_Signal=[0]
for i in range(1,len(data)):
    if data['MACD'].iloc[i]>data['Signal'].iloc[i] and data['MACD'].iloc[i-1]<data['Signal'].iloc[i-1]:
        S1_Signal.append(1)
    elif data['MACD'].iloc[i]<data['Signal'].iloc[i] and data['MACD'].iloc[i-1]>data['Signal'].iloc[i-1]:
        S1_Signal.append(-1)
    else:
        S1_Signal.append(0) 

In [289]:
data['S1_Signal']=pd.DataFrame(S1_Signal)

In [290]:
capital=10000
portfolio_value=[]
quantity=[]
entry=[]
exit=[]
new_signal=[]
signal=[]
trade_type=[]
current_trade_peak=0

Without stop loss and take profit

In [291]:

def Backtesting(data,cap):
    capital=cap
    portfolio_value=[]
    quantity=[]
    current_position=0
    holding=0
    current_portfolio_value=0
    entry=[]
    exit=[]
    new_signal=[]
    signal=list(data['S1_Signal'])
    trade_type=[]
    for i in range(len(data)-1):
      if capital<0:
         print('Capital vanished')
      if current_position==0:
         if signal[i]==0:
            current_position=0
            new_signal.append(0)
            holding=0
            portfolio_value.append(capital)
            quantity.append(0)
         elif signal[i]==1:
            current_position=1
            holding=capital/data['Close'].iloc[i]
            capital=0
            new_signal.append(1)
            quantity.append(holding)
            current_portfolio_value=holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            entry.append(i)
            trade_type.append('Long')
            current_trade_peak=current_portfolio_value
            # current_stop_loss=(1-stop_loss_percent)*current_trade_peak
            # current_take_profit=(1+take_profit_percent)*current_trade_peak

         elif signal[i]==-1:
            current_position=-1
            holding=capital/data['Close'].iloc[i]
            capital=2*capital
            new_signal.append(-1)
            quantity.append(holding)
            current_portfolio_value=capital-holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            entry.append(i)
            trade_type.append('Short')
            current_trade_peak=current_portfolio_value
            # current_stop_loss=(1+stop_loss_percent)*current_trade_peak
            # current_take_profit=(1-take_profit_percent)*current_trade_peak

      elif current_position==1:
         if signal[i]==0 or signal[i]==1:
            temp=holding*data['Close'].iloc[i]
            # if temp<current_stop_loss:
            #    stop_loss_count.append(i)
            #    current_position=0
            #    quantity.append(0)
            #    capital=holding*data['Close'].iloc[i]
            #    current_portfolio_value=capital
            #    portfolio_value.append(current_portfolio_value)
            #    new_signal.append(-1)
            #    exit.append(i)
            # elif temp>=current_take_profit:
            #    take_profit_count.append(i)
            #    current_position=0
            #    quantity.append(0)
            #    capital=holding*data['Close'].iloc[i]
            #    current_portfolio_value=capital
            #    portfolio_value.append(current_portfolio_value)
            #    new_signal.append(-1)
            #    exit.append(i)
            # elif holding*data['Close'].iloc[i]>current_trade_peak:
            #    current_trade_peak=holding*data['Close'].iloc[i]
            #    current_stop_loss=(1-stop_loss_percent)*current_trade_peak
            #    current_position=1
            #    quantity.append(holding)
            #    current_portfolio_value=holding*data['Close'].iloc[i]
            #    portfolio_value.append(current_portfolio_value)
            #    new_signal.append(0)
            current_position=1
            quantity.append(holding)
            current_portfolio_value=holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            new_signal.append(0)   
         else:
            current_position=0
            quantity.append(0)
            capital=holding*data['Close'].iloc[i]
            current_portfolio_value=capital
            portfolio_value.append(current_portfolio_value)
            new_signal.append(-1)
            exit.append(i)

      elif current_position==-1:
        if signal[i]==0 or signal[i]==-1:
         temp=capital-holding*data['Close'].iloc[i]
         #   if temp>current_stop_loss:
         #        stop_loss_count.append(i)
         #        current_position=0
         #        quantity.append(0)
         #        capital=capital-holding*data['Close'].iloc[i]
         #        current_portfolio_value=capital
         #        portfolio_value.append(current_portfolio_value)
         #        new_signal.append(1)
         #        exit.append(i)
         #   elif temp<=current_take_profit:
         #        take_profit_count.append(i)
         #        current_position=0
         #        quantity.append(0)
         #        capital=capital-holding*data['Close'].iloc[i]
         #        current_portfolio_value=capital
         #        portfolio_value.append(current_portfolio_value)
         #        new_signal.append(1)
         #        exit.append(i)
         #   elif (capital-holding*data['Close'].iloc[i])<current_trade_peak:
         #        current_trade_peak=capital-holding*data['Close'].iloc[i]
         #        current_stop_loss=(1+stop_loss_percent)*current_trade_peak
         #        current_position=-1
         #        quantity.append(holding)
         #        current_portfolio_value=capital-holding*data['Close'].iloc[i]
         #        portfolio_value.append(current_portfolio_value)
         #        new_signal.append(0)
         current_position=-1
         quantity.append(holding)
         current_portfolio_value=capital-holding*data['Close'].iloc[i]
         portfolio_value.append(current_portfolio_value)
         new_signal.append(0)
                    
        else:
           current_position=0
           capital=capital-holding*data['Close'].iloc[i]
           quantity.append(0)
           current_portfolio_value=capital
           portfolio_value.append(current_portfolio_value)
           new_signal.append(1)
           exit.append(i)    

    if current_position==1:
            current_position=0
            quantity.append(0)
            capital=holding*data['Close'].iloc[i]
            current_portfolio_value=capital
            portfolio_value.append(current_portfolio_value)
            new_signal.append(-1)
            exit.append(i)
    elif current_position==-1:
           current_position=0
           capital=capital-holding*data['Close'].iloc[i]
           quantity.append(0)
           current_portfolio_value=capital
           portfolio_value.append(current_portfolio_value)
           new_signal.append(1)
           exit.append(i)
    else:
            current_position=0
            new_signal.append(0)
            holding=0
            portfolio_value.append(capital)
            quantity.append(0)
          

    return capital,portfolio_value,quantity,entry,exit,new_signal,trade_type

capital,portfolio_value,quantity,entry,exit,new_signal,trade_type=Backtesting(data,capital)            

In [292]:
data['Portfolio']=portfolio_value

In [293]:
def MaxDrawDown(df):
    
    max_peak_till_now = df.cummax()
    drawdown = (df- max_peak_till_now)/max_peak_till_now
    return drawdown.min()

In [294]:
MaxDrawDown(data['Portfolio'])*100

-16.89887533355526

In [295]:
(capital-10000)/100

146.7976108601023

With stop loss and take profit

In [296]:
capital1=10000
portfolio_value1=[]
quantity1=[]
entry1=[]
exit1=[]
new_signal1=[]
signal1=[]
trade_type1=[]
current_trade_peak1=0
stop_loss_count1=[]
take_profit_count1=[]

In [297]:

def Backtesting1(data,cap,stop_p=0.0675,take_p=0.4):
    capital=cap
    portfolio_value=[]
    quantity=[]
    current_position=0
    holding=0
    current_portfolio_value=0
    current_stop_loss=0
    current_take_profit=0
    stop_loss_percent=stop_p
    take_profit_percent=take_p
    entry=[]
    exit=[]
    new_signal=[]
    signal=list(data['S1_Signal'])
    trade_type=[]
    current_trade_peak=0
    stop_loss_count=[]
    take_profit_count=[]
    for i in range(len(data)-1):
      if capital<0:
         print('Capital vanished')
      if current_position==0:
         if signal[i]==0:
            current_position=0
            new_signal.append(0)
            holding=0
            portfolio_value.append(capital)
            quantity.append(0)
         elif signal[i]==1:
            current_position=1
            holding=capital/data['Close'].iloc[i]
            capital=0
            new_signal.append(1)
            quantity.append(holding)
            current_portfolio_value=holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            entry.append(i)
            trade_type.append('Long')
            current_trade_peak=current_portfolio_value
            current_stop_loss=(1-stop_loss_percent)*current_trade_peak
            current_take_profit=(1+take_profit_percent)*current_trade_peak

         elif signal[i]==-1:
            current_position=-1
            holding=capital/data['Close'].iloc[i]
            capital=2*capital
            new_signal.append(-1)
            quantity.append(holding)
            current_portfolio_value=capital-holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            entry.append(i)
            trade_type.append('Short')
            current_trade_peak=current_portfolio_value
            current_stop_loss=(1+stop_loss_percent)*current_trade_peak
            current_take_profit=(1-take_profit_percent)*current_trade_peak

      elif current_position==1:
         if signal[i]==0 or signal[i]==1:
            temp=holding*data['Close'].iloc[i]
            if temp<current_stop_loss:
               stop_loss_count.append(i)
               current_position=0
               quantity.append(0)
               capital=holding*data['Close'].iloc[i]
               current_portfolio_value=capital
               portfolio_value.append(current_portfolio_value)
               new_signal.append(-1)
               exit.append(i)
            elif temp>=current_take_profit:
               take_profit_count.append(i)
               current_position=0
               quantity.append(0)
               capital=holding*data['Close'].iloc[i]
               current_portfolio_value=capital
               portfolio_value.append(current_portfolio_value)
               new_signal.append(-1)
               exit.append(i)
            # elif holding*data['Close'].iloc[i]>current_trade_peak:
            #    current_trade_peak=holding*data['Close'].iloc[i]
            #    current_stop_loss=(1-stop_loss_percent)*current_trade_peak
            #    current_position=1
            #    quantity.append(holding)
            #    current_portfolio_value=holding*data['Close'].iloc[i]
            #    portfolio_value.append(current_portfolio_value)
            #    new_signal.append(0)
            else:
               current_position=1
               quantity.append(holding)
               current_portfolio_value=holding*data['Close'].iloc[i]
               portfolio_value.append(current_portfolio_value)
               new_signal.append(0)   
         else:
            current_position=0
            quantity.append(0)
            capital=holding*data['Close'].iloc[i]
            current_portfolio_value=capital
            portfolio_value.append(current_portfolio_value)
            new_signal.append(-1)
            exit.append(i)

      elif current_position==-1:
        if signal[i]==0 or signal[i]==-1:
           temp=capital-holding*data['Close'].iloc[i]
           if temp>current_stop_loss:
                stop_loss_count.append(i)
                current_position=0
                quantity.append(0)
                capital=capital-holding*data['Close'].iloc[i]
                current_portfolio_value=capital
                portfolio_value.append(current_portfolio_value)
                new_signal.append(1)
                exit.append(i)
           elif temp<=current_take_profit:
                take_profit_count.append(i)
                current_position=0
                quantity.append(0)
                capital=capital-holding*data['Close'].iloc[i]
                current_portfolio_value=capital
                portfolio_value.append(current_portfolio_value)
                new_signal.append(1)
                exit.append(i)
         #   elif (capital-holding*data['Close'].iloc[i])<current_trade_peak:
         #        current_trade_peak=capital-holding*data['Close'].iloc[i]
         #        current_stop_loss=(1+stop_loss_percent)*current_trade_peak
         #        current_position=-1
         #        quantity.append(holding)
         #        current_portfolio_value=capital-holding*data['Close'].iloc[i]
         #        portfolio_value.append(current_portfolio_value)
         #        new_signal.append(0)
           else:
                current_position=-1
                quantity.append(holding)
                current_portfolio_value=capital-holding*data['Close'].iloc[i]
                portfolio_value.append(current_portfolio_value)
                new_signal.append(0)
                    
        else:
           current_position=0
           capital=capital-holding*data['Close'].iloc[i]
           quantity.append(0)
           current_portfolio_value=capital
           portfolio_value.append(current_portfolio_value)
           new_signal.append(1)
           exit.append(i)    

    if current_position==1:
            current_position=0
            quantity.append(0)
            capital=holding*data['Close'].iloc[i]
            current_portfolio_value=capital
            portfolio_value.append(current_portfolio_value)
            new_signal.append(-1)
            exit.append(i)
    elif current_position==-1:
           current_position=0
           capital=capital-holding*data['Close'].iloc[i]
           quantity.append(0)
           current_portfolio_value=capital
           portfolio_value.append(current_portfolio_value)
           new_signal.append(1)
           exit.append(i)
    else:
            current_position=0
            new_signal.append(0)
            holding=0
            portfolio_value.append(capital)
            quantity.append(0)
          

    return capital,portfolio_value,quantity,entry,exit,new_signal,trade_type,stop_loss_count,take_profit_count

capital1,portfolio_value1,quantity1,entry1,exit1,new_signal1,trade_type1,stop_loss_count1,take_profit_count1=Backtesting1(data,capital1)            

In [298]:
data['Portfolio1']=portfolio_value1

In [299]:
def MaxDrawDown1(df):
    
    max_peak_till_now = df.cummax()
    drawdown = (df- max_peak_till_now)/max_peak_till_now
    return drawdown.min()


In [300]:
MaxDrawDown1(data['Portfolio1'])*100

-20.59922002550662

In [301]:
(capital1-10000)/100

123.15877116096705

Only trailing stop loss

In [302]:
capital2=10000
portfolio_value2=[]
quantity2=[]
entry2=[]
exit2=[]
new_signal2=[]
signal2=[]
trade_type2=[]
current_trade_peak2=0
stop_loss_count2=[]

In [303]:

def Backtesting2(data,cap,stop_p=0.0675):
    capital=cap
    portfolio_value=[]
    quantity=[]
    current_position=0
    holding=0
    current_portfolio_value=0
    current_stop_loss=0
    stop_loss_percent=stop_p
    entry=[]
    exit=[]
    new_signal=[]
    signal=list(data['S1_Signal'])
    trade_type=[]
    current_trade_peak=0
    stop_loss_count=[]
    take_profit_count=[]
    for i in range(len(data)-1):
      if capital<0:
         print('Capital vanished')
      if current_position==0:
         if signal[i]==0:
            current_position=0
            new_signal.append(0)
            holding=0
            portfolio_value.append(capital)
            quantity.append(0)
         elif signal[i]==1:
            current_position=1
            holding=capital/data['Close'].iloc[i]
            capital=0
            new_signal.append(1)
            quantity.append(holding)
            current_portfolio_value=holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            entry.append(i)
            trade_type.append('Long')
            current_trade_peak=current_portfolio_value
            current_stop_loss=(1-stop_loss_percent)*current_trade_peak

         elif signal[i]==-1:
            current_position=-1
            holding=capital/data['Close'].iloc[i]
            capital=2*capital
            new_signal.append(-1)
            quantity.append(holding)
            current_portfolio_value=capital-holding*data['Close'].iloc[i]
            portfolio_value.append(current_portfolio_value)
            entry.append(i)
            trade_type.append('Short')
            current_trade_peak=current_portfolio_value
            current_stop_loss=(1+stop_loss_percent)*current_trade_peak

      elif current_position==1:
         if signal[i]==0 or signal[i]==1:
            temp=holding*data['Close'].iloc[i]
            if temp<current_stop_loss:
               stop_loss_count.append(i)
               current_position=0
               quantity.append(0)
               capital=holding*data['Close'].iloc[i]
               current_portfolio_value=capital
               portfolio_value.append(current_portfolio_value)
               new_signal.append(-1)
               exit.append(i)
            elif holding*data['Close'].iloc[i]>current_trade_peak:
               current_trade_peak=holding*data['Close'].iloc[i]
               current_stop_loss=(1-stop_loss_percent)*current_trade_peak
               current_position=1
               quantity.append(holding)
               current_portfolio_value=holding*data['Close'].iloc[i]
               portfolio_value.append(current_portfolio_value)
               new_signal.append(0)
            else:
               current_position=1
               quantity.append(holding)
               current_portfolio_value=holding*data['Close'].iloc[i]
               portfolio_value.append(current_portfolio_value)
               new_signal.append(0)   
         else:
            current_position=0
            quantity.append(0)
            capital=holding*data['Close'].iloc[i]
            current_portfolio_value=capital
            portfolio_value.append(current_portfolio_value)
            new_signal.append(-1)
            exit.append(i)

      elif current_position==-1:
        if signal[i]==0 or signal[i]==-1:
           temp=capital-holding*data['Close'].iloc[i]
           if temp>current_stop_loss:
                stop_loss_count.append(i)
                current_position=0
                quantity.append(0)
                capital=capital-holding*data['Close'].iloc[i]
                current_portfolio_value=capital
                portfolio_value.append(current_portfolio_value)
                new_signal.append(1)
                exit.append(i)
          
           elif (capital-holding*data['Close'].iloc[i])<current_trade_peak:
                current_trade_peak=capital-holding*data['Close'].iloc[i]
                current_stop_loss=(1+stop_loss_percent)*current_trade_peak
                current_position=-1
                quantity.append(holding)
                current_portfolio_value=capital-holding*data['Close'].iloc[i]
                portfolio_value.append(current_portfolio_value)
                new_signal.append(0)
           else:
                current_position=-1
                quantity.append(holding)
                current_portfolio_value=capital-holding*data['Close'].iloc[i]
                portfolio_value.append(current_portfolio_value)
                new_signal.append(0)
                    
        else:
           current_position=0
           capital=capital-holding*data['Close'].iloc[i]
           quantity.append(0)
           current_portfolio_value=capital
           portfolio_value.append(current_portfolio_value)
           new_signal.append(1)
           exit.append(i)    

    if current_position==1:
            current_position=0
            quantity.append(0)
            capital=holding*data['Close'].iloc[i]
            current_portfolio_value=capital
            portfolio_value.append(current_portfolio_value)
            new_signal.append(-1)
            exit.append(i)
    elif current_position==-1:
           current_position=0
           capital=capital-holding*data['Close'].iloc[i]
           quantity.append(0)
           current_portfolio_value=capital
           portfolio_value.append(current_portfolio_value)
           new_signal.append(1)
           exit.append(i)
    else:
            current_position=0
            new_signal.append(0)
            holding=0
            portfolio_value.append(capital)
            quantity.append(0)
          

    return capital,portfolio_value,quantity,entry,exit,new_signal,trade_type,stop_loss_count

capital2,portfolio_value2,quantity2,entry2,exit2,new_signal2,trade_type2,stop_loss_count2=Backtesting2(data,capital2)            

In [304]:
data['Portfolio2']=portfolio_value2

In [305]:
def MaxDrawDown2(df):
    
    max_peak_till_now = df.cummax()
    drawdown = (df- max_peak_till_now)/max_peak_till_now
    return drawdown.min()

In [306]:
MaxDrawDown2(data['Portfolio2'])*100

-13.526785577656094

In [307]:
(capital2-10000)/100

226.2165118081145